## Information Coding 2023-24

## Lab assignment 3 (Lab session 4, 16-11-2023)

## Public key cryptography

 (Estimated working time: 2 hours lab + 3 hours home);
	
 Student names:

 Buliga Theodor Ioan,
 
 Javier Cirugeda Bugallo

**INSTRUCTIONS**
 
- Practices are to be done in groups of two (or one). 
- This solved practice has to be uploaded in Moodle before September 30th, 23:55h. 
- The submission must be done by the first author (alphabetical order).
- Name the file with authors' names.
- This practice accounts for 5% of final grade for both autors. Grading items:
    * Defined functions 50% (eficiency, clarity, code documentation)
    * Exercise solving 40%
    * Mathematical correction, clarity of results, language precission, 10%.
- To get any credit, physical presence in the practice session is a must.



 

The following alphabet shall be used throughout this practice, unless otherwise stated.

In [40]:
ALFA=" ABCDEFGHIJKLMNÑOPQRSTUVWXYZ0123456789"
len(ALFA)

38

### 1 Numerical equivalent of a message

Remember that any message m written in an alphabet A (of size r) can be given a numerical value,
	the position of the message in the list of all messages, with lexicographical order. 
	
For instance, the message "ABCD", on the alphabet ALFA of length r=38, can be computed as

In [41]:
r=len(ALFA)
(ALFA.find("A")+1)*r**3+(ALFA.find("B")+1)*r**2+(ALFA.find("C")+1)*r+(ALFA.find("D")+1)


114233

Notice that ```string.find(char)``` gives the position of the the first appearance of char in string,
     from 0 to len(string)-1. We need position from 1 to len(string).

This polynomial in r can be computed more efficiently using the Horner scheme (Ruffini).

In [42]:
value=0
for c in "ABCD":
    value = value * r + (ALFA.find(c) + 1)
value

114233

#### 1.1 Exercise (2 pts)
(a) (0.75 pts) Define a function ```NumberEquivalent```:

**INPUT:**  a string m (a message), a string A (the alphabet)

**OUTPUT:** The numerical value of m on the alphabet A.
	
*(If a symbol of the message is not in A, the function must give an error message)*
    
(b) (0.75 pts) Define a function ```TextEquivalent```:

**INPUT:**  a natural number n (the numerical equivalent), a string A (the alphabet)

**OUTPUT:** a string with the message corresponding to the number and alphabet given.

(c) (0.5 pts) Try the function on some relevant cases. They must include the following:
	    
        Numerical value on the alphabet ALFA of:
        ""
        "L"
        "W"
        "AA"
        "BCD"
        "GCM41"
        "Patata" 
        
        Message equivalent of the numbers obtained above

In [43]:
def NumberEquivalent(m:str, A:str):
    if len(m) == 0:
        return 0
    result = 0
    for c in m:
        if c not in A:
            raise ValueError("The symbol " + c + " is not in the alphabet ")
        result = result * len(A) + (A.find(c) + 1)
    return result

print(NumberEquivalent("ABCD", ALFA))
print(NumberEquivalent("", ALFA))
print(NumberEquivalent("L", ALFA))
print(NumberEquivalent("W", ALFA))
print(NumberEquivalent("AA", ALFA))
print(NumberEquivalent("BCD", ALFA))
print(NumberEquivalent("GCM41", ALFA))
#print(NumberEquivalent("Patata", ALFA)) # Exception is raised because we have unknown values in the message that are not in the alphabet

114233
0
13
25
78
4489
16922076


In [44]:
def TextEquivalent(n:int, A:str):
    if n == 0:
        return ""
    result = ""
    while n > 0:
        result = A[((n - 1) % len(A))] + result
        n = (n - 1) // len(A)
    return result

print(TextEquivalent(114233, ALFA))
print(TextEquivalent(0, ALFA))
print(TextEquivalent(13, ALFA))
print(TextEquivalent(25, ALFA))
print(TextEquivalent(78, ALFA))
print(TextEquivalent(4489, ALFA))
print(TextEquivalent(16922076, ALFA))

ABCD

L
W
AA
BCD
GCM41


### 2 RSA cryptosystem
####  2.1 Exercise (4 pts)

(a) (1 pt) Write a function ```GenerateRSAKeys```.

**INPUT:**  an integer k (size of primes in digits)

**OUTPUT:** a list [n,e] where 
- n=pq, p,q being random primes of size k digits, with p>2q  
- e is a random integer, prime with (p-1)(q-1)

*After running the function, the global variable ```RSAPrivateKey``` must be equal to [n,d], where d is the modular inverse of e mod (p-1)(q-1)*
	
Use the function to generate Alice's private and public key, with prime numbers of 10 digits.
	
*Remark:*
- ```random.randrange(a, b)``` gives a random integer between a and b-1
- ```sympy.nextprime(n)``` gives the first prime number greater than n
- ```sympy.prevprime(n)``` gives the previous prime number to n

In [45]:
import random as rd
import math
import time
import sympy as sp
k=5
p=random.randrange(10**(k-1),10**k)
sp.nextprime(p)
# Function to generate RSA key pair
def GenerateRSAKeys(k):
    # Generate random prime numbers p and q with k digits
    p = sp.nextprime(random.randrange(10**(k-1), 10**k))
    q = sp.nextprime(random.randrange(10**(k-1), 10**k))
    
    # Ensure p is more than twice the value of q
    while p <= 2*q:
        p = sp.nextprime(random.randrange(10**(k-1), 10**k))
        q = sp.nextprime(random.randrange(10**(k-1), 10**k))
    
    # Calculate phi(n) and n
    phi = (p-1) * (q-1) 
    n = p * q

    # Generate a random number e and find its greatest common divisor with phi(n) and we ensure that e and phi(n) are coprime
    e = rd.randrange(2, phi)
    while sp.gcd(e, phi) != 1:
        e = rd.randrange(2, phi)
    
    # Calculate the modular multiplicative inverse of e modulo phi(n)
    d = sp.mod_inverse(e, phi)
    
    # Store the private key [n, d] in a global variable
    global RSAPrivateKey 
    RSAPrivateKey = [n, d]
    
    # Return the public key [n, e]
    return [n, e]
    
# Print the generated RSA public key with 10 digits and the Private Key
print("Public Key:", GenerateRSAKeys(10))
print("Private Key:", RSAPrivateKey)

Public Key: [18061769115052795969, 2006153267336180969]
Private Key: [18061769115052795969, 11958020243307173829]


(b) (2 pts) Define a function ```RSA```.

**INPUT:**  a string m (the message) OR a list of natural numbers (the cryptogram), a string A (the alphabet), a list key with two natural numbers (the public or private key, key=[n,e] OR [n,d])

**OUTPUT:**
- If m is a string, the output is the RSA encription of m on the alphabet A using the key [n,e] (the message is divided in blocks of length ```l = int(sp.log((r-1)*n+1)/sp.log(r)-1)```, gets the numerical value of each block, and encrypts it as $\text{value}^e \mod n$. The encryptions, in a list, are the output).
- If m is a list of naturals, the output is the string resulting of the decryption of the cryptogram on the alphabet A using the key [n,d] (each integer $k$ in the list is transformed as $k^d \mod n$; the resulting number is transformed into the corresponding message. The output is the concatenation  of all partial messages from the list)

In [46]:
# Function for RSA encryption and decryption
def RSA(m, A, key):
    # Check if the key is correct
    if len(key) != 2:
        raise ValueError("Incorrect key")
    if not all(isinstance(k, int) for k in key):
        raise TypeError("Values in key not integers")
    
    # Extract modulus (n) and exponent (e or d) from the key
    n, e = key

    # Calculate the block size (l) based on the alphabet size (r) and modulus (n)
    r = len(A)
    l = math.floor(math.log((r - 1) * n + 1, r) - 1)

    # If the input message is a string, perform encryption
    if isinstance(m, str):
        encryptions = []
        # Process the message in blocks of size 'l'
        for i in range(0, len(m), l):
            block = m[i:i+l]
            # Convert the block to a numerical representation using the given alphabet 'A'
            number = NumberEquivalent(block, A)
            # Perform modular exponentiation to encrypt the block
            encryptions.append(pow(number, e, n))
        return encryptions
    
    # If the input message is a list (encrypted blocks), perform decryption
    elif isinstance(m, list):
        decryptions = ""
        # Perform modular exponentiation on each encrypted block and convert back to text
        for number in m:
            decryptions += TextEquivalent(pow(number, key[1], key[0]), A)
        return decryptions
  
    else:
        # Raise an error for invalid input type
        raise ValueError("Invalid input for message")


# Example usage
# Define the message 'm'
m = "BUENAS TARDES PROFE"
print("Message: ", m)

# Encrypt the message using RSA with a 10-digit key and alphabet 'ALFA'
enc = RSA(m, ALFA, GenerateRSAKeys(10))
print("Encrypted message: ", enc)

# Decrypt the encrypted message using RSA and the private key
dec = RSA(enc, ALFA, RSAPrivateKey)
print("Decrypted message: ", dec)


Message:  BUENAS TARDES PROFE
Encrypted message:  [3615288089733553725, 1428579173737175793]
Decrypted message:  BUENAS TARDES PROFE


(c) (0.5 pts) Encrypt the message "PLEASE PAY 6500 EUROS TO SUPPLIER 32" to send it to
	a user with public key [222404123,479]. 
 
(d) (0.5 pts) Find out the private key of the user above. Measure the time needed to do it.
	Which size would be enough to assure reasonable safety if we use Maxima as cryptanalitic tool?

	For robust protection against cryptanalytic tools such as Maxima, it is advisable to use a key size of at least 2048 bits for RSA encryption. For enhanced security, a key size of 4096 bits is recommended.

In [47]:
# c)
encrypted_message = RSA("PLEASE PAY 6500 EUROS TO SUPPLIER 32", ALFA, [222404123, 479])
print("Encrypted message with public key [222404123,479] is: ",encrypted_message)

Encrypted message with public key [222404123,479] is:  [117405104, 70857741, 9645250, 31582339, 46415456, 108104541, 62747465, 15444253]


In [48]:
# d)
# Function to find the private key given the modulus (n) and public exponent (e)
def find_private_key(n, e):
    # Record the start time for measuring the execution time
    start_time = time.time()
    
    # Factorize the modulus 'n' to find prime factors 'p' and 'q'
    p, q = sp.factorint(n).keys()
    
    # Calculate Euler's totient function (phi_n)
    phi = (p - 1) * (q - 1)
    
    # Find the modular multiplicative inverse of 'e' modulo 'phi_n' to get the private exponent 'd'
    d = sp.mod_inverse(e, phi)
    
    # Record the end time and calculate the time taken for the computation
    end_time = time.time()
    time_taken = end_time - start_time
    
    # Return the private key 'd' and the time taken for the computation
    return d, time_taken

# Example public key [n, e]
public_key = [222404123, 479]

# Find the corresponding private key and measure the time taken
private_key, time_taken = find_private_key(*public_key)


# Print the decrypted message using the RSA function with the generated private key
print("Decrypted message: " + RSA(encrypted_message, ALFA, [222404123, private_key]))
print(f"Private key: {private_key}")
print(f"Time: {time_taken} seconds")

# For robust protection against cryptanalytic tools such as Maxima, it is advisable to use a key size of at least 2048 bits for RSA encryption. For enhanced security, a key size of 4096 bits is recommended.

Decrypted message: PLEASE PAY 6500 EUROS TO SUPPLIER 32
Private key: 63130271
Time: 0.0 seconds


### 3 ElGamal cryptosystem
To generate a pair of public/private keys for ElGamal, the user chooses a big prime $p$ and a generator $g$, and a (secret) random integer $a$ ($2\le a \le p-1)$, and computes $\beta= g^a \mod p$.

The public key is $[p,g,\beta]$ and the private key es $[p,g,a]$. So, $a$ is the discrete logarithm
of $\beta$ in base $g$ $\mod p$, and it is hard to compute from the public key.
  
####  3.1 Exercise (4pts)
(a) (1 pt) Define a function ```GenerateElGamalKeys```:

**INPUT:**  a positive integer n (size of key)

**OUTPUT:** a list [p,g,$\beta$], the public key ($p$ is a random prime of $k$ digits, $g$ is a random primitive root of $\mathbb{Z}_p$, and $\beta$ is $g^a \mod p$, where $a$ is a random integer such that $2\le a\le p-1$.

*After running the function, the global variable ```ElGamalPrivateKey``` must be equal to [p,g,a].*

Use it to generate Alice keys, of size 10 digits.

In [49]:
# Function to generate ElGamal public and private keys
def GenerateElGamalKeys(k):
    global ElGamalPrivateKey
    # Generate a random prime number 'p' with k digits
    
    # The bounds for the number
    lowerBound = 10 ** (k - 1) 
    upperBound = 10 ** k
    
    # Generate the number
    p = sp.nextprime(random.randrange(lowerBound, upperBound))
    
    # Find a primitive root 'g' modulo 'p'
    g = sp.primitive_root(p)
    
    # Choose a random private key 'a' in the range [2, p-1]
    lowerBound = 2
    upperBound = p - 1
    
    # Calculate a
    a = random.randrange(lowerBound, upperBound)

    # Calculate the public key 'beta' using the formula beta = g^a mod p
    publicKey = pow(g, a, p)

    # Store the private key [p, g, a] in a global variable
    ElGamalPrivateKey = [p, g, a]

    # Return the public key [p, g, beta]
    return [p, g, publicKey]

# Generate ElGamal public key and store the private key in a global variable
size = 10
public_key = GenerateElGamalKeys(size)

# Print the generated public and private keys
print(f"Alice's Public Key: {public_key}")
print(f"Alice's Private Key: {ElGamalPrivateKey}")

Alice's Public Key: [6260661343, 3, 3838476648]
Alice's Private Key: [6260661343, 3, 4425701941]


	
To send a message $m$ ($0\le m<p$) to a user B with public key $[p, g, \beta]$:
- Choose a (secret) random integer $b$
- Compute $r=g^b \mod p$
- Compute $t=m \cdot \beta^b \mod p$ 
- Transmit $(r,t)$.

To decrypt the message, B (whose private key is $[n,g,a]$) does $m= t \cdot r^{-a} \mod p$.

*REMARK:* $b$ must be different for each block of each message. The use of a fixed value of $b$ for, say, $m_1$ and $m_2$, would give the same value of $r$. The cryptograms would be $(r,t_1)$,$(r,t_2)$.
If an Enemy gets the plaintext of $m_1$, then he/she would also have $m_2$, because $t_1/m_1=t_2/m_2$.

(b) (2 pts) Write a function ```ElGamal```:

**INPUT:**  a string m OR a list of pairs of integers (the plaintext OR the cryptogram);
a string A (the alphabet); a list of three integers (the public OR the private key)

**OUTPUT:** If m is a string, the output is the ElGamal encription of m on the alphabet A             using the key [n,g,$\beta$] (the message is divided in blocks of length ```l = int(sp.log((r-1)*n+1)/sp.log(r)-1)```), where r is the length of the alphabet, gets the numerical value of each block, and encrypts it. The encryptions, in a list, are the output).

If m is a list of naturals, the output is the string resulting of the decryption of the cryptogram on the alphabet A using the key [n,g,a].

In [50]:
# Function for ElGamal encryption and decryption
def ElGamal(m, A, key):
    # Length of the alphabet
    r = len(A)

    # Encryption if m is a string
    if isinstance(m, str):
        n, g, beta = key
        l = math.floor(math.log((r - 1) * n + 1, r) - 1)  # Block size

        # Check characters in the message
        if any(char not in A for char in m):
            raise ValueError("Wrong char in message")

        # Convert the message to numerical blocks
        numerical_blocks = [NumberEquivalent(m[i:i + l], A) for i in range(0, len(m), l)]

        # Encrypt each numerical block
        ciphertext = []
        for block in numerical_blocks:
            b = random.randint(2, n - 2)
            r = pow(g, b, n)
            t = (block * pow(beta, b, n)) % n
            ciphertext.append([r, t])

        return ciphertext

    # Decryption if m is a list of integers
    elif isinstance(m, list) and all(isinstance(pair, list) and len(pair) == 2 for pair in m):
        n, g, a = key
        decrypted_blocks = []

        for pair in m:
            r, t = pair

            if not (isinstance(r, int) and isinstance(t, int)):
                raise TypeError("Blocks are corrupted")

            s = pow(r, a, n)
            s_inv = pow(s, -1, n)
            decrypted_block = (t * s_inv) % n
            decrypted_blocks.append(decrypted_block)

        decrypted_message = "".join(TextEquivalent(block, A) for block in decrypted_blocks)
        return decrypted_message

    else:
        raise ValueError("Wrong input...")
    
# Example usage
# Define the message 'm'
message = "BUENAS NOCHES PROFE"
print("Message: ", message)

# Generate ElGamal public key and retrieve the private key
public_key = GenerateElGamalKeys(10)
private_key = ElGamalPrivateKey

# Encrypt the message using ElGamal with the generated public key and alphabet 'ALFA'
encrypted_message = ElGamal(message, ALFA, public_key)
print("Encrypted Message: ", encrypted_message)

# Decrypt the encrypted message using ElGamal and the private key
decrypted_message = ElGamal(encrypted_message, ALFA, private_key)
print("Decrypted Message: ", decrypted_message)

Message:  HOLA
Encrypted Message:  [[415132833, 1085744323]]
Decrypted Message:  HOLA


(c) (2 pts) Decrypt the following cryptogram, received by a user with public key [50707, 3, 20546],
	with the alphabet ALFA2=" ABCDEFGHIJKLMNÑOPQRSTUVWXYZ,.;:" 
    
	(The plaintext is a cite from the Diccionario Filosófico, by F. Savater)	

In [51]:
ALFA2=" ABCDEFGHIJKLMNÑOPQRSTUVWXYZ,.;:" 
cryptogram=[[41749, 32388], [36791, 19828], [45776, 32978], [15223, 31030],[22602, 16456], [43960, 49512], [15431, 39994], [18335, 15467], 
  [35728, 18942], [24173, 23040], [50026, 47396], [46109, 47136],[42094, 596], [36082, 2297], [28445, 28761], [38987, 6869], 
  [2346, 22153], [448, 48244], [3902, 39939], [19263, 8169],[16317, 26356], [43486, 21203], [45467, 23581], [43061, 14317], 
  [21000, 2678], [43301, 16569], [965, 41145], [34068, 16157],[21610, 606], [48214, 34768], [42091, 28687], [3107, 33107], 
  [11306, 46425], [41831, 31840], [29514, 40852], [11330, 35006],[9196, 14169], [16205, 35270], [49938, 7374], [14905, 33802], 
  [3426, 2403], [30329, 27771], [20503, 20741], [1762, 14362],[5476, 41000], [31942, 43965], [28215, 21648], [37604, 26282], 
  [25221, 47356], [11866, 14540], [29417, 34879], [23346, 14085],[469, 48803], [11742, 29350], [1703, 13846], [46715, 26597], 
  [16836, 18003], [37115, 40072], [38685, 5592], [47746, 2881],[40055, 43352], [35459, 34393], [38742, 47536], [25208, 19396], 
  [12502, 23595], [48785, 40472], [38027, 7349], [22562, 32186],[14110, 21511], [22203, 17718], [29352, 520], [36627, 35648], 
  [19873, 44441], [25477, 773], [12266, 31034], [28462, 31177],[29890, 36847], [15888, 48876], [5641, 4554], [48429, 32418], 
  [407, 30842], [43254, 25844], [34446, 37898], [28732, 933],[30431, 1148], [25150, 39800], [34057, 18937], [44968, 37939], 
  [9931, 46273], [21119, 35129], [10278, 32635], [18645, 1680],[15287, 3665], [47589, 23592], [8023, 26387], [29218, 26865], 
  [5336, 43857], [17307, 40727], [13523, 50329], [8532, 4680],[37637, 49075], [22012, 18934], [45192, 39824], [17919, 31147], 
  [48304, 43235], [8462, 48690], [8819, 17372], [9087, 4082],[44937, 6858], [27874, 9151], [4069, 45131], [48680, 776], 
  [45804, 22308], [11439, 12906], [48395, 7349], [2634, 1375],[46272, 45515], [10907, 11098], [16463, 3719], [5639, 20735], 
  [44223, 22217], [34002, 931], [18459, 1848], [23168, 12741],[40053, 43898], [15086, 31746], [30292, 23409], [10095, 23157], 
  [43064, 6147], [21365, 30548], [25664, 32252], [12631, 13443],[30384, 38405], [20117, 43159], [31244, 21528], [19843, 30977],[19582, 41518], [9690, 3548], [33669, 18742], [7022, 36942],[7721, 33179], [8950, 18928], [2457, 31131], [33704, 46534], 
  [14898, 14474], [15337, 21403], [28736, 3197], [12414, 13027],[44450, 16046], [38650, 33836], [1574, 9660], [44250, 41302],[50150, 37561], [40878, 6826], [1692, 39055], [40090, 23501], 
  [49230, 33453], [42833, 24904], [13234, 41351], [26262, 7715],[39320, 24127], [26036, 21319], [46294, 47512], [37481, 47820],
  [21500, 29838], [8375, 41325], [19475, 33675], [20811, 13390],[17806, 39670], [49177, 863], [42615, 11141], [37421, 39871], 
  [46988, 23651], [15389, 2771], [6367, 31630], [46822, 43185],[27212, 8007], [3837, 38892], [12462, 85], [12958, 27], 
  [2736, 1178], [9427, 6506], [11070, 17074], [29299, 800],[18048, 17002], [24589, 49925], [9985, 6503], [38918, 32893], 
  [49599, 10585], [23690, 3983], [44282, 18843], [23246, 27752],[1854, 28762], [4815, 31991], [2896, 35533], [31776, 41314], 
  [4165, 6099], [3965, 18193], [18515, 49989], [31938, 27655],[42292, 1886], [44883, 18493], [23948, 7026], [29749, 948], 
  [37931, 24286], [6118, 24690], [1464, 11531], [12216, 19963],[4205, 28328], [42977, 48703], [35822, 46982], [13739, 22064], 
  [29099, 49268], [4933, 10564], [39931, 9504], [12406, 14241],[22157, 6650], [43024, 30307], [39654, 737], [26450, 41399], 
  [18370, 48608], [32222, 34257], [12546, 43546], [32258, 2171],[42620, 2868], [25474, 14780], [27242, 4634], [5934, 13973], 
  [17774, 46741], [36116, 18833], [50194, 31908], [22575, 15297],[720, 49787], [24045, 4214], [16944, 41620], [45842, 24285], 
  [23362, 22652], [30105, 27695], [1868, 14623], [46542, 5807], [13140, 48952], [47156, 39080], [47336, 45131], [18417, 10418], 
  [38598, 48405], [13935, 46625], [9112, 9791], [32447, 12922],[35826, 44903], [18872, 25864], [30800, 27078], [20058, 18314], 
  [35610, 42893], [160, 16832], [5805, 22970], [46876, 28007],[31813, 36510], [22138, 37263], [6235, 42006], [37709, 18086], 
  [42844, 33286], [37608, 26270], [34327, 4293], [31874, 22439],[42364, 40066], [47511, 19178], [20113, 7455], [38958, 45295],
  [10081, 6996]]
len(cryptogram)

269

In [52]:
a = sp.discrete_log(50707, 20546, 3)
print(a)

decrypted_message = ElGamal(cryptogram, ALFA2, [50707, 3, a])
print(f"Decrypted Message: {decrypted_message}")

8057
Decrypted Message: PARTAMOS DE LA BASE DE QUE TODA ACCION HUMANA TIENE COMO OBJETIVO CONSEGUIR ALGO VENTAJOSO PARA EL AGENTE QUE LA REALIZA. SEGUN CIPOLLA, PUEDEN ESTABLECERSE CUATRO CATEGORIAS MORALES: PRIMERO ESTAN LOS BUENOS, CUYAS ACCIONES LOGRAN VENTAJAS PARA SI MISMOS Y PARA LOS DEMAS; DESPUES VIENEN LOS INCAUTOS, QUE PRETENDEN OBTENER VENTAJAS PARA SI MISMOS PERO EN REALIDAD LO QUE HACEN ES PROPORCIONARSELAS A OTROS; MAS ABAJO QUEDAN LOS MALOS, QUE OBTIENEN BENEFICIOS A COSTA DEL DAÑO DE OTROS; Y POR ULTIMO ESTAN LOS ESTUPIDOS QUE, PRETENDAN SER BUENOS O MALOS, LO UNICO QUE CONSIGUEN A FIN DE CUENTAS ES PERJUICIOS TANTO PARA ELLOS COMO PARA LOS DEMAS. HAY MUCHOS MAS ESTUPIDOS QUE BUENOS MALOS O INCAUTOS. ADEMAS EL ESTUPIDO ES PEOR QUE EL MALO YA QUE EL MALO DESCANSA DE VEZ EN CUANDO PERO EL ESTUPIDO JAMAS.
